In [ ]:
# Bellow code is tested with the following GPUs: L4

In [1]:
import sys
print(sys.version)

3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]


In [2]:
!nvidia-smi

Mon Aug 18 06:43:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   34C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# In the Terminal run (~10 min):
# git clone https://github.com/sgl-project/sglang
# cd sglang
# pip3 install pip --upgrade
# pip3 install -e "python[all]"

In [12]:
# check SGLang version
import importlib.metadata
print(importlib.metadata.version('sglang'))

0.5.0rc2


0.5.0rc2


In [ ]:
# In the Terminal run:
# python3 -m sglang.launch_server --model openai/gpt-oss-20b

# Wait until Web API is fully started
# ~10 minutes for the first start (includes model download)
# ~3 minutes for the second start

In [13]:
# Quick connection test
import requests

response = requests.get("http://localhost:30000/get_model_info")
response.json()


{'model_path': 'openai/gpt-oss-20b',
 'tokenizer_path': 'openai/gpt-oss-20b',
 'is_generation': True,
 'preferred_sampling_params': None,
 'weight_version': 'default'}

In [14]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:30000/v1",
    api_key="EMPTY"
)

In [24]:
system_prompt = """
You are a multiclass multilabel conversation topic classifier.
Reasoning effort is low.
Your answer must contain only the class names.
The only allowed classes are: phone, tv, headphone, internet.
"""

In [ ]:
user_content = """
Klientas: Laba, ieškau naujo telefono. Mano senas vos pusdienį laiko bateriją.

Asistentas: Supratau! Pradėkime nuo to, kas jums svarbiausia – baterijos veikimo laikas, kamera, našumas ar kas nors kito?

Klientas: Tikrai baterija. Nemėgstu krauti telefono du kartus per dieną.

Asistentas: Puiku, tuomet rekomenduočiau XPhone 14 Max – jis turi 5000 mAh bateriją, kuri įprastai naudojant laiko iki dviejų dienų. Be to, palaiko 45W greitąjį įkrovimą, tad nuo 0% iki 50% įkrausite per mažiau nei 30 minučių.

Klientas: Skamba gerai. O kaip su kamera? Daug fotografuoju socialiniams tinklams.

Asistentas: Šis modelis turi trigubą objektyvų sistemą: 50 MP pagrindinį jutiklį, 12 MP itin plataus kampo ir 10 MP teleobjektyvą. Puikiai fotografuoja prasto apšvietimo sąlygomis, o su AI stabilizacija vaizdo įrašai bus sklandūs.

Klientas: Mhm, o ekranas? Daug žiūriu „Netflix“.

Asistentas: Tai 6,7 colio AMOLED ekranas su 120 Hz atnaujinimo dažniu ir HDR10+ palaikymu – vaizdas bus ryškus ir itin sklandus.

Klientas: O kaip su atmintimi? Nenoriu vis trinti nuotraukų.

Asistentas: Galite rinktis iš 128 GB, 256 GB arba 512 GB. Jei mėgstate laikyti daug vaizdo įrašų, 256 GB būtų optimalus pasirinkimas.

Klientas: Supratau. Kiek kainuoja?

Asistentas: 256 GB versija kainuoja 899 €. Bet šiuo metu turime komplektą – telefonas ir mūsų aukščiausios klasės belaidės ausinės už 949 €.

Klientas: Ausinės?

Asistentas: Taip! XSound Pro belaidės ausinės turi aktyvų triukšmo slopinimą, iki 30 valandų veikimo ir puikiai dera su XPhone garso lustu. Įprastai jos kainuoja 199 €, tad su komplektu sutaupote 150 €. Ir įsivaizduokite „Netflix“ su kino salės garso kokybe.

Klientas: Gundo... Ar būna juodos spalvos?

Asistentas: Taip – juodos, sidabrinės ir riboto leidimo rožinio aukso.

Klientas: Gerai, imu telefoną su ausinėmis.

Asistentas: Puikus pasirinkimas! Paruošiu užsakymą, kad galėtumėte iškart džiaugtis naujuoju įrenginiu.
"""

In [42]:
# ~ 4 seconds for classification
result = client.chat.completions.create(
    model="openai/gpt-oss-20b",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_content}
    ]
)

print(result.choices[0].message.content)

analysisWe need to classify conversation topics. The conversation covers phone and headphone. It's multi-label, so output both. Allowed classes: phone, tv, headphone, internet. So output "phone, headphone".assistantfinalphone, headphone


In [22]:
result

ChatCompletion(id='2c286a0b5fd24918a128d6cc4f9ae2c2', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='analysisWe need to determine the class labels among phone, tv, headphone, internet. The conversation is about a phone and headphones. The user wants a phone; assistant also mentions headphones. So could be both phone and headphone. Multi-label. So output should contain "phone" and "headphone". Just those class names.\n\nWe must output only class names, presumably comma-separated? The instruction: "Your answer must contain only the class names." No formatting. So maybe just "phone, headphone" or each on new line? Up to interpretation. Likely comma separated. We\'ll output "phone, headphone".assistantfinalphone, headphone', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning_content=None), matched_stop=200002)], created=1755500844, model='openai/gpt-oss-20b', object='chat.complet

In [36]:
# At the moment of writing SGLang is unable to properly parse final answer
# Instead it returns full reasoning and answer mixed.
result.choices[0].message.content.split("assistantfinal")[1]

'phone, headphone'

In [38]:
user_content_2 = """
Klientas: Sveiki. Skambinu, nes pastaruoju metu mano interneto ryšys labai prastas. Dažnai dingsta, o greitis daug mažesnis nei sutartyje numatyta.

Operatorius: Sveiki. Atsiprašome už nepatogumus. Ar galėtumėte pasakyti, ar ryšys dingsta tam tikru metu, ar tai vyksta nuolat?

Klientas: Dažniausiai vakare, bet kartais ir dienos metu. Labai sunku naudotis, ypač kai reikia dirbti iš namų.

Operatorius: Supratau. Patikrinsiu jūsų linijos būklę sistemoje. Prašau minutėlę palaukti... Gerai, matau, kad jūsų vietovėje šiuo metu vykdomi tinklo atnaujinimo darbai, todėl gali pasitaikyti trikdžių.

Klientas: O kiek dar tai truks? Nes jau savaitė, kai neįmanoma normaliai naudotis internetu.

Operatorius: Remiantis informacija, darbai turėtų būti baigti per artimiausias dvi dienas. Po jų ryšio kokybė turėtų pagerėti.

Klientas: O ar už tą laikotarpį bus pritaikyta kokia nors kompensacija?

Operatorius: Taip, žinoma. Galime pritaikyti nuolaidą už šį mėnesį dėl patirtų nepatogumų.

Klientas: Gerai, dėkoju. Labai tikiuosi, kad situacija pagerės.

Operatorius: Dar kartą atsiprašome už nepatogumus. Jei trikdžiai nesibaigs po darbų pabaigos, kviečiu susisiekti dar kartą, ir išsiųsime meistrą patikrinti įrangos.

Klientas: Supratau, ačiū už pagalbą.

Operatorius: Ačiū, kad kreipėtės. Linkiu gražios dienos!
"""

In [57]:
# ~ 4 seconds for classification
result = client.chat.completions.create(
    model="openai/gpt-oss-20b",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_content_2},
    ],
)

print(result.choices[0].message.content)

analysisWe need to classify the conversation into one or more classes: phone, tv, headphone, internet. The conversation is about internet connection issues. So class is internet. No other topics. So output "internet".assistantfinalinternet


In [58]:
# At the moment of writing SGLang is unable to properly parse final answer
# Instead it returns full reasoning and answer mixed.
result.choices[0].message.content.split("assistantfinal")[1]

'internet'

In [59]:
!nvidia-smi

Mon Aug 18 07:43:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   48C    P0             28W /   72W |   21673MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----